In [2]:
Cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [41]:
!pip install plotly -q
import pandas as pd
import requests


#Transformation de la liste des villes en un dataframe où on pourra ajouter latitude et longitude
df = pd.DataFrame(Cities)
df = df.rename(columns={0: 'Cities'})

In [42]:
#Boucle permettant de trouver les coordonnées à partir du nom de ville

df['Cities'] = df['Cities'].str.replace(' ','+')

for i in df.index:
    city = df.loc[i,'Cities']
    position = requests.get('https://nominatim.openstreetmap.org/search?q={}&format=json'.format(city)).json()
    df.loc[i,'latitude'] = position[0]['lat']
    df.loc[i,'longitude'] = position[0]['lon']

df['Cities'] = df['Cities'].str.replace('+',' ')   
df.head()

,Cities,latitude,longitude
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


In [43]:
#Boucle permettant de récupérer humidité + Température afin de les ajouter dans 7 nouvelles colonnes chacune
#Plus l'humidité est élevé plus le risque de pluie et de temps nuageux est important

for i in df.index:
    latit = df.loc[i,'latitude']
    longit = df.loc[i,'longitude']
    data = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&exclude=current,minutely,hourly&appid=d9c40c5f887d643e52d7f4b81197d9a8'.format(latit,longit))
    df.loc[i,'Météo j1'] = data.json()['daily'][0]['temp']['day']
    df.loc[i,'Météo j2'] = data.json()['daily'][1]['temp']['day']
    df.loc[i,'Météo j3'] = data.json()['daily'][2]['temp']['day']
    df.loc[i,'Météo j4'] = data.json()['daily'][3]['temp']['day']
    df.loc[i,'Météo j5'] = data.json()['daily'][4]['temp']['day']
    df.loc[i,'Météo j6'] = data.json()['daily'][5]['temp']['day']
    df.loc[i,'Météo j7'] = data.json()['daily'][6]['temp']['day']
    df.loc[i,'Météo j8'] = data.json()['daily'][7]['temp']['day']



    df.loc[i,'Humidité j1'] = data.json()['daily'][0]['humidity']
    df.loc[i,'Humidité j2'] = data.json()['daily'][1]['humidity']
    df.loc[i,'Humidité j3'] = data.json()['daily'][2]['humidity']
    df.loc[i,'Humidité j4'] = data.json()['daily'][3]['humidity']
    df.loc[i,'Humidité j5'] = data.json()['daily'][4]['humidity']
    df.loc[i,'Humidité j6'] = data.json()['daily'][5]['humidity']
    df.loc[i,'Humidité j7'] = data.json()['daily'][6]['humidity']
    df.loc[i,'Humidité j8'] = data.json()['daily'][7]['humidity']
    
df.head()

,Cities,latitude,longitude,Météo j1,Météo j2,Météo j3,Météo j4,Météo j5,Météo j6,Météo j7,Météo j8,Humidité j1,Humidité j2,Humidité j3,Humidité j4,Humidité j5,Humidité j6,Humidité j7,Humidité j8
0,Mont Saint Michel,48.6359541,-1.511459954959514,6.65,8.15,8.52,8.57,8.47,9.15,9.94,10.05,81.0,97.0,70.0,67.0,73.0,95.0,79.0,77.0
1,St Malo,48.649518,-2.0260409,8.35,11.44,9.66,9.22,8.91,11.61,9.49,10.27,74.0,95.0,62.0,62.0,73.0,91.0,82.0,71.0
2,Bayeux,49.2764624,-0.7024738,5.65,7.04,7.47,7.74,6.38,6.69,9.74,9.85,74.0,91.0,76.0,66.0,81.0,92.0,74.0,79.0
3,Le Havre,49.4938975,0.1079732,7.23,6.48,8.12,7.28,5.49,6.77,9.93,8.93,59.0,90.0,75.0,76.0,72.0,90.0,74.0,82.0
4,Rouen,49.4404591,1.0939658,5.12,3.52,8.58,5.14,4.59,4.58,9.43,7.55,73.0,93.0,76.0,91.0,68.0,91.0,71.0,84.0


In [46]:
#Export des données en CSV
df.to_csv('Geographie_des_villes.csv')
df = pd.read_csv('Geographie_des_villes.csv', decimal=".")

In [47]:
#Transposition du dataframe afin de pouvoir realiser des graph (on aura donc 16 fois plus de lignes mais 14 colonnes en moins)
df_transposed = df.melt(id_vars=["Cities", "latitude","longitude"],value_vars=["Météo j1", "Météo j2", "Météo j3", "Météo j4", "Météo j5", "Météo j6", "Météo j7", "Météo j8"] ,var_name = "Jour", value_name = "Degrés")
df_transposed2 = df.melt(id_vars=["Cities", "latitude","longitude"],value_vars=["Humidité j1", "Humidité j2", "Humidité j3", "Humidité j4", "Humidité j5", "Humidité j6", "Humidité j7", "Humidité j8"] ,var_name = "Humidité", value_name = "Taux")

display(df_transposed.head(5))
display(df_transposed2.head(5))

,Cities,latitude,longitude,Jour,Degrés
0,Mont Saint Michel,48.635954,-1.511460,Météo j1,6.65
1,St Malo,48.649518,-2.026041,Météo j1,8.35
2,Bayeux,49.276462,-0.702474,Météo j1,5.65
3,Le Havre,49.493898,0.107973,Météo j1,7.23
4,Rouen,49.440459,1.093966,Météo j1,5.12


,Cities,latitude,longitude,Humidité,Taux
0,Mont Saint Michel,48.635954,-1.511460,Humidité j1,81.0
1,St Malo,48.649518,-2.026041,Humidité j1,74.0
2,Bayeux,49.276462,-0.702474,Humidité j1,74.0
3,Le Havre,49.493898,0.107973,Humidité j1,59.0
4,Rouen,49.440459,1.093966,Humidité j1,73.0


In [48]:
#On fait un join des 2 dataframe pour avoir toutes nos infos en un dataframe
df2 = df_transposed[['Cities','latitude','longitude', 'Jour','Degrés']].join(df_transposed2['Taux']).rename(columns={3: 'Jour'})
df2['Jour'] = df2['Jour'].str.replace('Météo', '')
df2.head(5)

,Cities,latitude,longitude,Jour,Degrés,Taux
0,Mont Saint Michel,48.635954,-1.511460,j1,6.65,81.0
1,St Malo,48.649518,-2.026041,j1,8.35,74.0
2,Bayeux,49.276462,-0.702474,j1,5.65,74.0
3,Le Havre,49.493898,0.107973,j1,7.23,59.0
4,Rouen,49.440459,1.093966,j1,5.12,73.0


In [52]:
#Meteo les plus chaudes par jour (7 prochains jours)

import plotly.express as px
import plotly.io as pio
pio.renderers.default="iframe_connected"

df2['latitude'] = df2['latitude'].replace('.', ',')
df2['longitude'] = df2['longitude'].replace('.', ',')

mask = (df2['Degrés'] > 5) & (df2['Taux'] < 50) #Il faut remplacer la valeur minimale en fonction de la période
data_viz = df2.loc[mask,:]

fig = px.scatter_mapbox(data_viz, lat="latitude", lon="longitude", color="Degrés",  mapbox_style="carto-positron", animation_frame="Jour",size="Degrés", size_max = 15,
                        color_continuous_scale=px.colors.diverging.Portland,zoom=0 ,hover_name='Cities')
fig.show()

In [54]:
#Risques de pluie (en rouge la ou le risque est faible, en bleu la où il est important)

import plotly.express as px
import plotly.io as pio
pio.renderers.default="iframe_connected"

df2['latitude'] = df2['latitude'].replace('.', ',')
df2['longitude'] = df2['longitude'].replace('.', ',')


fig = px.scatter_mapbox(df2, lat="latitude", lon="longitude", color="Taux",  mapbox_style="carto-positron", animation_frame="Jour",size="Taux", size_max = 10,
                        color_continuous_scale=px.colors.diverging.Portland[::-1],zoom=4)
fig.show()

In [57]:
#Export du dataframe
df2.to_csv('Final_file.csv')

In [ ]:
#Meteo par jour (en rouge là ou il fait chaud et en bleu là où il fait froid)

import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
pio.renderers.default="iframe_connected"

df['latitude'] = df['latitude'].replace('.', ',')
df['longitude'] = df['longitude'].replace('.', ',')


fig = px.scatter_mapbox(df2, lat="latitude", lon="longitude", color="Degrés",  mapbox_style="carto-positron", animation_frame="Jour",size="Degrés", size_max = 15,
                        color_continuous_scale=px.colors.diverging.Portland,zoom=4)
fig.show()

In [56]:
#Aller à Kayak2 pour consulter la suite